In [2]:
import numpy as np
import pandas as pd

In [3]:
raw_data_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_data_raw_19_apr.xlsx"
raw_data_df = pd.read_excel(raw_data_path,"Raw Data")
#raw_data_df.head(5)

,Prefix,Item code,Master item prefix,Master item code,Warehouse name,Warehouse group name,Description,Alt. group,Buyer code,Item group 1,...,Head of repl.,Supplier repl. info.,Replaced according to supplier,Non-guaranteed returns enabled override,Guaranteed returns enabled override,Warehouse free text 1,Warehouse free text 2,Warehouse free text 3,Warehouse free text 4,Warehouse free text 5
0,ZAW,-0005-LONG,ZAW,-0005-LONG,Capetown,Babcock Equipment,BEACON LIGHT BRACKET,NaN,NaN,OBSOLETE,...,No,NaN,No,Undefined,Undefined,NaN,NaN,NaN,NaN,NaN
1,PJ,023-105-S,PJ,023-105-S,Bartlett,Babcock Equipment,O-RING,NaN,NaN,729,...,No,NaN,No,Undefined,Undefined,NaN,NaN,NaN,NaN,NaN
2,PJ,023-105-S,PJ,023-105-S,Bloemfontein,Babcock Equipment,O-RING,NaN,NaN,729,...,No,NaN,No,Undefined,Undefined,NaN,NaN,NaN,NaN,NaN
3,PJ,023-105-S,PJ,023-105-S,Capetown,Babcock Equipment,O-RING,NaN,NaN,729,...,No,NaN,No,Undefined,Undefined,NaN,NaN,NaN,NaN,NaN
4,PJ,023-105-S,PJ,023-105-S,Durban,Babcock Equipment,O-RING,NaN,NaN,729,...,No,NaN,No,Undefined,Undefined,NaN,NaN,NaN,NaN,NaN


In [4]:
ids_df = pd.read_excel(raw_data_path,"Interesting items")
ids_df.columns = ["item code"]

In [25]:
# Picking out one item
item_code = ids_df["item code"].loc[0]
one_item_df  = raw_data_df[raw_data_df['Item code'] == item_code]
one_item_df.head(3)

,Prefix,Item code,Master item prefix,Master item code,Warehouse name,Warehouse group name,Description,Alt. group,Buyer code,Item group 1,...,Head of repl.,Supplier repl. info.,Replaced according to supplier,Non-guaranteed returns enabled override,Guaranteed returns enabled override,Warehouse free text 1,Warehouse free text 2,Warehouse free text 3,Warehouse free text 4,Warehouse free text 5
104,SA,1030-61460,SA,1030-61460,Bartlett,Babcock Equipment,FILTER ELEMENT,NaN,NaN,717,...,No,0,No,Undefined,Undefined,NaN,NaN,NaN,NaN,NaN
105,SA,1030-61460,SA,1030-61460,Bloemfontein,Babcock Equipment,FILTER ELEMENT,NaN,NaN,717,...,No,0,No,Undefined,Undefined,NaN,NaN,NaN,NaN,NaN
106,SA,1030-61460,SA,1030-61460,Capetown,Babcock Equipment,FILTER ELEMENT,NaN,NaN,717,...,No,0,No,Undefined,Undefined,NaN,NaN,NaN,NaN,NaN


In [44]:
input_df = pd.DataFrame(columns=["Installation id", "Type", "Name", "Transport time", "Q", 
            "Unit cost", "Target item fill rate", "Demand type",	
            "Demand mean per time unit", "Demand stdev per time unit","Demand type","Stocked"])
input_df["Name"] = one_item_df["Warehouse name"]
input_df["Installation id"] = one_item_df["Warehouse name"]
input_df["Transport time"] = one_item_df["Lead time"]
input_df["Q"] = one_item_df["Constr. opt. OQ"]
input_df["Unit cost"] = one_item_df["Unit cost"]
input_df["Demand mean per time unit"] = one_item_df["EOD"]
input_df["Demand stdev per time unit"] = one_item_df["Std. dev."]
input_df["Demand type"] = one_item_df["Demand type"]
input_df["Stocked"] = one_item_df["Stocked"]

type_list = []
for name in input_df["Name"]:
    if name == "Johannesburg":
        type_list.append("RDC")
    else:
        type_list.append("Dealer")
input_df["Type"] = type_list

target_IFR_list = []
for target_rate in one_item_df["Target srv. lvl."]:
    if target_rate is not np.nan:
        target_fill_rate = (1/100)*float(target_rate[0:4])
    else:
        target_fill_rate = 0
    target_IFR_list.append(target_fill_rate)
input_df["Target item fill rate"] = target_IFR_list

input_df["Demand distribution"] = "Empiric_Compound_Poisson"

#Fixing indexes
input_df.index = range(0,len(input_df["Installation id"]))

,Installation id,Type,Name,Transport time,Q,Unit cost,Target item fill rate,Demand type,Demand mean per time unit,Demand stdev per time unit,Demand type,Stocked,Demand distribution
0,Bartlett,Dealer,Bartlett,5,11.0,140.09,0.980,Erratic,6.7120,3.9006,Erratic,Yes,Empiric_Compound_Poisson
1,Bloemfontein,Dealer,Bloemfontein,5,4.0,140.09,0.990,Fast,1.6960,1.0616,Fast,Yes,Empiric_Compound_Poisson
2,Capetown,Dealer,Capetown,5,9.0,140.09,0.980,Fast,5.3271,2.9309,Fast,Yes,Empiric_Compound_Poisson
3,Durban,Dealer,Durban,5,7.0,140.09,0.980,Fast,3.6960,3.1209,Fast,Yes,Empiric_Compound_Poisson
4,George,Dealer,George,5,3.0,140.09,0.900,Slow,0.4167,0.5149,Slow,Yes,Empiric_Compound_Poisson
5,Kimberley,Dealer,Kimberley,5,7.0,140.09,0.995,Fast,1.0688,1.2772,Fast,Yes,Empiric_Compound_Poisson
6,Middelburg,Dealer,Middelburg,5,2.0,140.09,0.900,Slow,0.2727,0.4671,Slow,Yes,Empiric_Compound_Poisson
7,Nelspruit,Dealer,Nelspruit,5,5.0,140.09,0.990,Erratic,1.3920,1.6400,Erratic,Yes,Empiric_Compound_Poisson
8,Port Elizabeth,Dealer,Port Elizabeth,5,5.0,140.09,0.995,Fast,0.9824,1.3234,Fast,Yes,Empiric_Compound_Poisson
9,Richards Bay,Dealer,Richards Bay,5,4.0,140.09,0.990,Fast,1.5458,1.8792,Fast,Yes,Empiric_Compound_Poisson


## Final script
Here we will iterate through the list and save everything in csv and excel-files.

In [51]:
# Initiating excel-writer
excel_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_inputs.xlsx"
writer = pd.ExcelWriter(excel_path)

for item_code in ids_df["item code"]:
    one_item_df  = raw_data_df[raw_data_df['Item code'] == item_code]
    input_df = pd.DataFrame(columns=["Installation id", "Type", "Name", "Transport time", "Q", 
                "Unit cost", "Target item fill rate", "Demand type",	
                "Demand mean per time unit", "Demand stdev per time unit","Demand type","Stocked"])
    input_df["Name"] = one_item_df["Warehouse name"]
    input_df["Installation id"] = one_item_df["Warehouse name"]
    input_df["Transport time"] = one_item_df["Lead time"]
    input_df["Q"] = one_item_df["Constr. opt. OQ"]
    input_df["Unit cost"] = one_item_df["Unit cost"]
    input_df["Demand mean per time unit"] = one_item_df["EOD"]
    input_df["Demand stdev per time unit"] = one_item_df["Std. dev."]
    input_df["Demand type"] = one_item_df["Demand type"]
    input_df["Stocked"] = one_item_df["Stocked"]

    type_list = []
    for name in input_df["Name"]:
        if name == "Johannesburg":
            type_list.append("RDC")
        else:
            type_list.append("Dealer")
    input_df["Type"] = type_list

    target_IFR_list = []
    for target_rate in one_item_df["Target srv. lvl."]:
        if target_rate is not np.nan:
            target_fill_rate = (1/100)*float(target_rate[0:4])
        else:
            target_fill_rate = 0
        target_IFR_list.append(target_fill_rate)
    input_df["Target item fill rate"] = target_IFR_list

    input_df["Demand distribution"] = "Empiric_Compound_Poisson"

    #Fixing indexes
    input_df.index = range(0,len(input_df["Installation id"]))

    # Saving a csv
    google_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/csv_files"
    csv_path = f"{google_path}/item_{item_code}_input.csv"
    input_df.to_csv(csv_path)

    # Writing to excel
    excel_sheet_name = f"item_{item_code}_input"
    input_df.to_excel(writer, sheet_name = excel_sheet_name) 

writer.save()